# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397865300482                   -0.90    5.0   24.5ms
  2   -8.400242950387       -2.62       -1.74    1.0   17.8ms
  3   -8.400405102446       -3.79       -2.97    1.5   18.8ms
  4   -8.400427840399       -4.64       -2.97    3.2   54.8ms
  5   -8.400427905174       -7.19       -3.01    1.0   18.4ms
  6   -8.400428149272       -6.61       -4.44    1.0   18.7ms
  7   -8.400428155749       -8.19       -4.41    2.2   22.2ms
  8   -8.400428156251       -9.30       -5.22    1.0   18.9ms
  9   -8.400428156275      -10.62       -6.53    1.2   19.3ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397772680049                   -0.90           5.0    622ms
  2   -8.400381604026       -2.58       -1.79   0.80    2.0    623ms
  3   -8.400422151621       -4.39       -3.02   0.80    1.0    198ms
  4   -8.400428105347       -5.23       -3.42   0.80    2.5   19.4ms
  5   -8.400428152419       -7.33       -4.56   0.80    1.0   15.7ms
  6   -8.400428156249       -8.42       -5.35   0.80    2.2   19.6ms
  7   -8.400428156276      -10.57       -6.55   0.80    2.0   18.5ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

┌ Warning: x_tol is deprecated. Use x_abstol or x_reltol instead. The provided value (-1) will be used as x_abstol.
└ @ Optim ~/.julia/packages/Optim/8dE7C/src/types.jl:110
┌ Warning: f_tol is deprecated. Use f_abstol or f_reltol instead. The provided value (-1) will be used as f_reltol.
└ @ Optim ~/.julia/packages/Optim/8dE7C/src/types.jl:120
n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.997956043364                   -1.04    3.11s
  2   -1.341872910571        0.37       -0.69    115ms
  3   -4.344656862808        0.48       -0.43   39.0ms
  4   -5.752156965889        0.15       -0.53   70.7ms
  5   -7.278399890699        0.18       -0.78   39.5ms
  6   -7.413752934656       -0.87       -1.39   30.8ms
  7   -7.983533223286       -0.24       -1.48   29.1ms
  8   -8.221987658946       -0.62       -1.55   57.4ms
  9   -8.301103420641       -1.10       -1.62   30.7ms
 10   -8.356483030175       -1.26       -2.06   29

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397834513131                   -0.90    5.0   24.7ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427965486                   -1.78    26.4s
  2   -8.400428156277       -6.72       -4.00    2.08s
  3   -8.400428156277      -14.75       -7.78   94.7ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 5.402882769486723e-7
|ρ_newton - ρ_scfv| = 2.453357458693683e-7
|ρ_newton - ρ_dm|   = 1.4568376815205163e-6
